# 봄/여름 모델링 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import numpy as np
import random
import os
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from catboost import CatBoostRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import RobustScaler
from scipy.signal import savgol_filter

from sklearn.model_selection import train_test_split
from pygam import LinearGAM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [ ]:
# 데이터 읽어오기
data_na = pd.read_csv("C:/Users/shn20/Desktop/FILES/기상청/지면온도 데이터/surface_tp_train.csv")
data_imp = pd.read_csv("C:/Users/shn20/Desktop/FILES/기상청/지면온도 데이터/지면온도train_imputed.csv")

# 분석에 이용할 데이터 만들기 
data_na['surface_tp_train.mmddhh']=data_na['surface_tp_train.mmddhh'].astype('str')

data_na['시간']=data_na['surface_tp_train.mmddhh'].str[-2:]
data_na['일']=data_na['surface_tp_train.mmddhh'].str[-4:-2]
data_na['월']=data_na['surface_tp_train.mmddhh'].str[:-4]

data_na.rename(columns={'surface_tp_train.stn':'지점번호'},inplace=True)


data=pd.concat([data_na[['지점번호','시간','월','일']],
                data_imp],axis=1)

data=data.drop(['time'],axis=1).dropna(axis=0)
need_scale=['평균기온', '평균이슬점온도', '평균상대습도', 
         '평균풍속', '누적강수량', '누적강수유무']

data[['월','일','시간']]=data[['월','일','시간']].astype('int')

encoder=LabelEncoder()
data['현천계현천']=encoder.fit_transform(data['현천계현천'])

# 계절별로 나누어주기

spring=data[data['월'].isin([2,3,4])] ## 봄: 2-4월 
summer=data[data['월'].isin([5,6,7])] ## 여름: 5-7월
fall=data[data['월'].isin([8,9,10])] ## 가을: 8-10월
winter=data[data['월'].isin([11,12,1])] ## 겨울: 11-1월

    
# RobustScaler
spscaler = RobustScaler() 
suscaler = RobustScaler() 
fascaler = RobustScaler()
wiscaler = RobustScaler()


# Robust Scaling 적용
spring[need_scale] = spscaler.fit_transform(spring[need_scale])
summer[need_scale] = suscaler.fit_transform(summer[need_scale])
fall[need_scale] = fascaler.fit_transform(fall[need_scale])
winter[need_scale] = wiscaler.fit_transform(winter[need_scale])

train_spring=spring
train_summer= summer
train_fall = fall 
train_winter = winter 

In [ ]:
# 데이터 읽어오기
test_na = pd.read_csv("C:/Users/shn20/Desktop/FILES/기상청/지면온도 데이터/surface_tp_test.csv")
test_imp = pd.read_csv("C:/Users/shn20/Desktop/FILES/기상청/지면온도 데이터/지면온도test_imputed.csv")

# 분석에 이용할 데이터 만들기 
test_na['surface_tp_test.mmddhh']=test_na['surface_tp_test.mmddhh'].astype('str')

test_na['시간']=test_na['surface_tp_test.mmddhh'].str[-2:]
test_na['일']=test_na['surface_tp_test.mmddhh'].str[-4:-2]
test_na['월']=test_na['surface_tp_test.mmddhh'].str[:-4]

test_na.rename(columns={'surface_tp_test.stn':'STN'},inplace=True)
test_na.rename(columns={'surface_tp_test.year':'YEAR'},inplace=True)
test_na.rename(columns={'surface_tp_test.mmddhh':'MMDDHH'},inplace=True)

test_na[['STN','YEAR','MMDDHH']]=test_na[['STN','YEAR','MMDDHH']].astype('str')  # 답안지 데이터와의 type 통일 

test=pd.concat([test_na[['STN','시간','월','일','YEAR','MMDDHH']],
                test_imp],axis=1)

test=test.drop(['time'],axis=1)

need_scale=['평균기온', '평균이슬점온도', '평균상대습도', 
         '평균풍속', '누적강수량', '누적강수유무']

test[['월','일','시간']]=test[['월','일','시간']].astype('int')

test['현천계현천']=encoder.transform(test['현천계현천'])

# 계절별로 나누어주기

test_spring=test[test['월'].isin([2,3,4])] ## 봄: 2-4월 
test_summer=test[test['월'].isin([5,6,7])] ## 여름: 5-7월
test_fall=test[test['월'].isin([8,9,10])] ## 가을: 8-10월
test_winter=test[test['월'].isin([11,12,1])] ## 겨울: 11-1월

# Robust Scaling 적용
test_spring[need_scale] = spscaler.transform(test_spring[need_scale])
test_summer[need_scale] = suscaler.transform(test_summer[need_scale])
test_fall[need_scale] = fascaler.transform(test_fall[need_scale])
test_winter[need_scale] = wiscaler.transform(test_winter[need_scale])

In [ ]:
x_col=data.drop(['지점번호','지면온도','적설깊이'],axis=1).columns.to_list() # 겨울의 경우 적설깊이 변수를 써줘야 한다. 
x_winter=data.drop(['지점번호','지면온도'],axis=1).columns.to_list()

y_col='지면온도'
print(x_col)

In [ ]:
def cv_MAE(data,model,x_col=x_col):
    
    MAE= np.array([])
    
    for i in tqdm(range(1,11)):
        idx=(data['지점번호']==i)
        
        train,test = data[~idx],data[idx]
        x_train,y_train = train[x_col],train[y_col]
        x_test,y_test = test[x_col],test[y_col]
    
        model.fit(x_train,y_train)
        y_pred=model.predict(x_test)
    
        score=mean_absolute_error(y_test,y_pred)
        MAE=np.append(MAE,score)

    print("폴드 별 MAE",MAE)
    print("CV 평균 MAE",np.mean(MAE))
    
def score(data,model,x_col=x_col):
    
    MAE= np.array([])
    
    for i in tqdm(range(1,11)):
        idx=(data['지점번호']==i)
        
        train,test = data[~idx],data[idx]
        x_train,y_train = train[x_col],train[y_col]
        x_test,y_test = test[x_col],test[y_col]
    
        model.fit(x_train,y_train)
        y_pred=model.predict(x_test)
    
        score=mean_absolute_error(y_test,y_pred)
        MAE=np.append(MAE,score)

    return(np.mean(MAE))

In [ ]:
class regressor:
    def __init__(self, model1=None, model2=None):
        if model1 is None:
            self.model1 = CatBoostRegressor(verbose=False,has_time=True,random_state=6)
        else:
            self.model1 = model1

        if model2 is None:
            self.model2 = lgb.LGBMRegressor(random_state=6)
        else:
            self.model2 = model2

        self.model3 = None
        self.model4 = None

    def fit(self, x_train, y_train):
        # 1차 적합
        self.model1.fit(x_train,y_train)
        train_fitted_values = self.model1.predict(x_train)
        train_residual = y_train - train_fitted_values

        # 2차 적합
        self.model2.fit(x_train, train_residual)
        train_fitted_values2 = self.model2.predict(x_train)
        train_residual2 = train_residual - train_fitted_values2

        # 3차 적합
        self.model3 = lgb.LGBMRegressor(random_state=6)
        self.model3.fit(x_train, train_residual2)
        train_fitted_values3 = self.model3.predict(x_train)
        train_residual3 = train_residual2 - train_fitted_values3

        # 4차 적합
        self.model4 = lgb.LGBMRegressor(random_state=6)
        self.model4.fit(x_train, train_residual3)

    def predict(self, x_test):
        test_fitted_values = self.model1.predict(x_test)
        test_fitted_values2 = self.model2.predict(x_test)
        test_fitted_values3 = self.model3.predict(x_test)
        test_residual = self.model4.predict(x_test)

        return test_fitted_values + test_fitted_values2 + test_fitted_values3 + test_residual

    def evaluate(self, x_test, y_test):
        y_pred = self.predict(x_test)
        score = mean_absolute_error(y_test, y_pred)
        return score

In [ ]:
# 봄 

sp_param1= {'learning_rate': 0.04158500359538149, 'depth': 12, 'iterations': 419}
spring1=CatBoostRegressor(verbose=False,**sp_param1,has_time=True,random_state=6)

sp_param2 = {'max_depth': 15, 'learning_rate': 0.09950693202165563, 'n_estimators': 902, 'min_child_samples': 59, 'subsample': 0.8052368021730489}
spring2= lgb.LGBMRegressor(**sp_param2,random_state=6) 

# 여름 

su_param1= {'learning_rate': 0.12515542897654602, 'depth': 13, 'iterations': 401} 
summer1=CatBoostRegressor(verbose=False,**su_param1,has_time=True,random_state=6)

su_param2= {'max_depth': 14, 'learning_rate': 0.09990601824979606, 'n_estimators': 924, 'min_child_samples': 74, 'subsample': 0.30422223766873524}
summer2= lgb.LGBMRegressor(**su_param2,random_state=6) 

In [ ]:
spr = regressor(model1=spring1,model2=spring2) #봄 
sumr= regressor(model1=summer1,model2=summer2) #여름 

spr.fit(train_spring[x_col],train_spring[y_col])
spring_predict=spr.predict(test_spring[x_col])

sumr.fit(train_summer[x_col],train_summer[y_col])
summer_predict=sumr.predict(test_summer[x_col])

## 옵튜나를 이용한 하이퍼 파라미터 튜닝 

In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

In [ ]:
# 봄 모델 1차 적합 모델 튜닝 
def objective_cbr(trial):
    
    
    # 하이퍼파라미터 검색 범위
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.5)
    depth = trial.suggest_int('depth', 7, 16)
    iterations = trial.suggest_int('iterations', 250, 500)

    model = CatBoostRegressor(verbose=False,
                              learning_rate=learning_rate,
                              depth=depth,
                              iterations=iterations)
    
    MAE= np.array([])
    
    data=spring
    
    for i in range(1,11):
        idx=(data['지점번호']==i)
        
        train,test = data[~idx],data[idx]
        x_train,y_train = train[x_col],train[y_col]
        x_test,y_test = test[x_col],test[y_col]
    
        model.fit(x_train,y_train)
        y_pred=model.predict(x_test)
    
        score=mean_absolute_error(y_test,y_pred)
        MAE=np.append(MAE,score)
        
    mae= np.mean(MAE)
    
    
    return mae

study = optuna.create_study(
        direction='minimize', 
        sampler=TPESampler())

study.optimize(objective_cbr,n_trials=15)

trial = study.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

# Best Trial: score 1.6421336515585658,
# params {'learning_rate': 0.04158500359538149, 'depth': 12, 'iterations': 419}

In [ ]:
# 봄 모델 2차 적합 모델 튜닝 

x_train, x_test, y_train, y_test = train_test_split(
    spring[x_col], spring[y_col], test_size=0.33,
    shuffle=True)

spring1.fit(x_train,y_train)
train_fitted_values = spring1.predict(x_train)
train_residual = y_train - train_fitted_values

test_fitted_values= spring1.predict(x_test)
test_residual = y_test - test_fitted_values

def objective_spring(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 10, 17),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 900, 970),
        'min_child_samples': trial.suggest_int('min_child_samples', 55, 75),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0)
    }

    model2 = lgb.LGBMRegressor(**param)
    model2.fit(x_train, train_residual)

    train_predictions2 = model2.predict(x_train)
    train_residuals2 = train_residual - train_predictions2
    y_pred = model2.predict(x_test)
    score = mean_absolute_error(test_residual, y_pred)

    return score

#study = optuna.create_study(direction='minimize')
study.optimize(objective_spring, n_trials=100)

best_params = study.best_params
print(best_params)

# {'max_depth': 12, 'learning_rate': 0.08334925654486108, 'n_estimators': 960, 'min_child_samples': 67, 'subsample': 0.5200456584507223}
# score= 1.3836848170174547
# {'max_depth': 15, 'learning_rate': 0.09950693202165563, 'n_estimators': 902, 'min_child_samples': 59, 'subsample': 0.8052368021730489}
# 1.378931408387576.


In [ ]:
# 여름 모델 1차 적합 모델 튜닝 

def objective_cbr(trial):
    
    
    # 하이퍼파라미터 검색 범위
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.5)
    depth = trial.suggest_int('depth', 7, 16)
    iterations = trial.suggest_int('iterations', 250, 500)

    model = CatBoostRegressor(verbose=False,
                              learning_rate=learning_rate,
                              depth=depth,
                              iterations=iterations)
    
    MAE= np.array([])
    
    data=summer
    
    for i in range(1,11):
        idx=(data['지점번호']==i)
        
        train,test = data[~idx],data[idx]
        x_train,y_train = train[x_col],train[y_col]
        x_test,y_test = test[x_col],test[y_col]
    
        model.fit(x_train,y_train)
        y_pred=model.predict(x_test)
    
        score=mean_absolute_error(y_test,y_pred)
        MAE=np.append(MAE,score)
        
    mae= np.mean(MAE)
    
    
    return mae

study = optuna.create_study(
        direction='minimize', 
        sampler=TPESampler())

study.optimize(objective_cbr,n_trials=5)

trial = study.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

# {'learning_rate': 0.12515542897654602, 'depth': 13, 'iterations': 401} 
# Best is trial 1 with value: 1.8528896916820217.

In [ ]:
# 여름 모델 2차 적합 모델 튜닝 

x_train, x_test, y_train, y_test = train_test_split(
    summer[x_col], summer[y_col], test_size=0.33,
    shuffle=True)

summer1.fit(x_train,y_train)
train_fitted_values = summer1.predict(x_train)
train_residual = y_train - train_fitted_values

test_fitted_values= summer1.predict(x_test)
test_residual = y_test - test_fitted_values

def objective_summer(trial):
    
    param = {
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 900, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 70, 80),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0)
    }

    model2 = lgb.LGBMRegressor(**param)
    model2.fit(x_train, train_residual)

    train_predictions2 = model2.predict(x_train)
    train_residuals2 = train_residual - train_predictions2
    y_pred = model2.predict(x_test)
    score = mean_absolute_error(test_residual, y_pred)

    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective_summer, n_trials=100)

best_params = study.best_params
print(best_params)

# {'max_depth': 14, 'learning_rate': 0.09990601824979606, 'n_estimators': 924, 'min_child_samples': 74, 'subsample': 0.30422223766873524}
# 1.5166825339675678.

# 가을 모델링 

In [ ]:
# Library
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import missingno as msno

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder , RobustScaler
from sklearn.metrics import mean_absolute_error

## CV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold

## Model
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor , HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb

## Tunning
import re
import optuna
from optuna.integration import XGBoostPruningCallback
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_fin = pd.read_csv('/content/drive/MyDrive/기상청/코드/이지윤/데이터/보간_수정_train.csv',encoding = 'UTF8')
test_fin = pd.read_csv('/content/drive/MyDrive/기상청/코드/이지윤/데이터/보간_수정_test.csv',encoding = 'UTF8')

In [ ]:
# train data

# Label Encoding
encoder = LabelEncoder()
encoder.fit(train_fin['year'])
train_fin["year"] = encoder.transform(train_fin['year']) + 1

# month / day / hour
train_fin['mmddhh'] = train_fin['mmddhh'].astype('str')
train_fin['hour'] = train_fin['mmddhh'].str[-2:]
train_fin['day'] = train_fin['mmddhh'].str[-4:-2]
train_fin['month'] = train_fin['mmddhh'].str[:-4]

train_fin['datetime'] = '000' + train_fin['year'].astype(str) + '-' + train_fin['month'] + '-' + train_fin['day'] + ' ' + train_fin['hour'] + ':00:00'
train_fin.drop(['mmddhh','year'], axis = 1 , inplace = True)

train_fin['month'] = train_fin['month'].astype('int')

# Y 에 Na 존재하는거 제거
train_fin['month'] = train_fin['month'].astype(int)
train_fin.dropna(subset=['ts'], inplace = True)

# 불필요 열 제거 및 인덱스 설정
train_fin.drop(['hour','day','Unnamed: 0'], axis = 1 , inplace = True)
train_fin = train_fin.reset_index(drop = True).set_index('datetime')

# test data

# Label Encoding
encoder = LabelEncoder()
encoder.fit(test_fin['year'])
test_fin["year"] = encoder.transform(test_fin['year']) + 1

encoder = LabelEncoder()
encoder.fit(test_fin['stn'])
test_fin['stn'] = encoder.transform(test_fin['stn']) + 1

# month / day / hour
test_fin['mmddhh'] = test_fin['mmddhh'].astype('str')
test_fin['hour'] = test_fin['mmddhh'].str[-2:]
test_fin['day'] = test_fin['mmddhh'].str[-4:-2]
test_fin['month'] = test_fin['mmddhh'].str[:-4]

test_fin['datetime'] = '000' + test_fin['year'].astype(str) + '-' + test_fin['month'] + '-' + test_fin['day'] + ' ' + test_fin['hour'] + ':00:00'

test_fin.drop(['Unnamed: 0','mmddhh','year','hour','day','month'], axis = 1 , inplace = True)
test_fin = test_fin.reset_index(drop = True).set_index('datetime')

## 지점별 적합

### 지점 1

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn1 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 1].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn1.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn1.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn1 = pd.get_dummies(data=surface_tp_train_hybrid_stn1, drop_first=True)

# surface_tp_train_hybrid_stn1 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 1].reset_index(drop=True)

# # 로버스트 스케일링
# train_X_num = surface_tp_train_hybrid_stn1[['ta', 'td', 'hm', 'ws', 'rn', 're', 'si',
#        'ss', 'sn']]

# # scikit-learn 패키지의 RobustScaler 클래스를 불러옵니다.
# from sklearn.preprocessing import RobustScaler
# # RobustScaler 객체를 생성합니다.
# robustScaler = RobustScaler()

# # fit_transform()을 사용해서 학습과 스케일링을 한 번에 적용합니다.
# X_train_robust = robustScaler.fit_transform(train_X_num)

# surface_tp_train_hybrid_stn1[['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn']] = X_train_robust

# start_date = "1900-02-01 00:00:00"  # 시작 날짜
# end_date = "1905-01-31 23:00:00"  # 종료 날짜
# freq = "H"  # 월 단위로 설정

# surface_tp_train_hybrid_stn1.index = pd.date_range(start=start_date, end=end_date, freq=freq)

# surface_tp_train_hybrid_stn1.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

# surface_tp_train_hybrid_stn1 = pd.get_dummies(data=surface_tp_train_hybrid_stn1, drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn1[f_col],
                                                    surface_tp_train_hybrid_stn1[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

In [ ]:
# from statsmodels.tsa.deterministic import DeterministicProcess
# from sklearn.linear_model import LinearRegression

# dp = DeterministicProcess(
#                           index = train_x.index,
#                           constant = True,
#                           order = 2,
#                           drop = True,
#                           )

# X = dp.in_sample()
# lm = LinearRegression(fit_intercept=False)
# lm.fit(X, train_y)

# test_const = [1 for i in range(test_x.shape[0])]
# test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
# X_ = pd.DataFrame([test_const, test_trend, test_trend_squared]).T
# X_.columns = ['const', 'trend', 'trend_squared']
# X_.index = test_x.index

# trend_train = lm.predict(X)
# trend_test = lm.predict(X_)

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm1 = LinearRegression(fit_intercept=False)
lm1.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm1.predict(X)
trend_test = lm1.predict(X_)

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [ ]:
# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

In [ ]:
def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best1 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best1.fit(train_x, train_y_delta)

xgb_train = xgb_best1.predict(train_x)
xgb_test = xgb_best1.predict(test_x)

plt.plot(surface_tp_train_hybrid_stn1.index, surface_tp_train_hybrid_stn1.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, xgb_train, label="train")

plt.plot(test_x.index, xgb_test, label="train")

In [ ]:
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

plt.plot(surface_tp_train_hybrid_stn1.index, surface_tp_train_hybrid_stn1.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, original_train, label="train")

plt.plot(test_x.index, original_test, label="train")

### 지점2

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn2 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 2].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn2.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn2.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn2 = pd.get_dummies(data=surface_tp_train_hybrid_stn2, drop_first=True)

In [ ]:
# surface_tp_train_hybrid_stn2 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 2].reset_index(drop=True)

# # 로버스트 스케일링
# train_X_num = surface_tp_train_hybrid_stn2[['ta', 'td', 'hm', 'ws', 'rn', 're', 'si',
#        'ss', 'sn']]

# # scikit-learn 패키지의 RobustScaler 클래스를 불러옵니다.
# from sklearn.preprocessing import RobustScaler
# # RobustScaler 객체를 생성합니다.
# robustScaler = RobustScaler()

# # fit_transform()을 사용해서 학습과 스케일링을 한 번에 적용합니다.
# X_train_robust = robustScaler.fit_transform(train_X_num)

# surface_tp_train_hybrid_stn2[['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn']] = X_train_robust

# start_date = "1900-02-01 00:00:00"  # 시작 날짜
# end_date = "1905-01-31 23:00:00"  # 종료 날짜
# freq = "H"  # 월 단위로 설정

# surface_tp_train_hybrid_stn2.index = pd.date_range(start=start_date, end=end_date, freq=freq)

# surface_tp_train_hybrid_stn2.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

# surface_tp_train_hybrid_stn2 = pd.get_dummies(data=surface_tp_train_hybrid_stn2, drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn2[f_col],
                                                    surface_tp_train_hybrid_stn2[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

In [ ]:
# from statsmodels.tsa.deterministic import DeterministicProcess
# from sklearn.linear_model import LinearRegression

# dp = DeterministicProcess(
#                           index = train_x.index,
#                           constant = True,
#                           order = 2,
#                           drop = True,
#                           )

# X = dp.in_sample()
# lm = LinearRegression(fit_intercept=False)
# lm.fit(X, train_y)

# test_const = [1 for i in range(test_x.shape[0])]
# test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
# X_ = pd.DataFrame([test_const, test_trend, test_trend_squared]).T
# X_.columns = ['const', 'trend', 'trend_squared']
# X_.index = test_x.index

# trend_train = lm.predict(X)
# trend_test = lm.predict(X_)

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm2 = LinearRegression(fit_intercept=False)
lm2.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm2.predict(X)
trend_test = lm2.predict(X_)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn2.index, surface_tp_train_hybrid_stn2.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, trend_train, label="train")

plt.plot(test_x.index, trend_test, label="train")

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [ ]:
# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

In [1]:
def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

NameError: name 'Trial' is not defined

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best2 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best2.fit(train_x, train_y_delta)

xgb_train = xgb_best2.predict(train_x)
xgb_test = xgb_best2.predict(test_x)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn2.index, surface_tp_train_hybrid_stn2.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, xgb_train, label="train")

plt.plot(test_x.index, xgb_test, label="train")

In [ ]:
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

In [ ]:
plt.plot(surface_tp_train_hybrid_stn2.index, surface_tp_train_hybrid_stn2.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, original_train, label="train")

plt.plot(test_x.index, original_test, label="train")

### 지점3 

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn3 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 3].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn3.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn3.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn3 = pd.get_dummies(data=surface_tp_train_hybrid_stn3, drop_first=True)

from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn3[f_col],
                                                    surface_tp_train_hybrid_stn3[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm3 = LinearRegression(fit_intercept=False)
lm3.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm3.predict(X)
trend_test = lm3.predict(X_)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn3.index, surface_tp_train_hybrid_stn3.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, trend_train, label="train")

plt.plot(test_x.index, trend_test, label="train")

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [ ]:
# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

In [ ]:
def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best3 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best3.fit(train_x, train_y_delta)

xgb_train = xgb_best3.predict(train_x)
xgb_test = xgb_best3.predict(test_x)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn3.index, surface_tp_train_hybrid_stn3.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, xgb_train, label="train")

plt.plot(test_x.index, xgb_test, label="train")

In [ ]:
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

In [ ]:
plt.plot(surface_tp_train_hybrid_stn3.index, surface_tp_train_hybrid_stn3.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, original_train, label="train")

plt.plot(test_x.index, original_test, label="train")

### 지점3

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn4 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 4].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn4.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn4.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn4 = pd.get_dummies(data=surface_tp_train_hybrid_stn4, drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn4[f_col],
                                                    surface_tp_train_hybrid_stn4[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm4 = LinearRegression(fit_intercept=False)
lm4.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm4.predict(X)
trend_test = lm4.predict(X_)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn4.index, surface_tp_train_hybrid_stn4.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, trend_train, label="train")

plt.plot(test_x.index, trend_test, label="train")

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [ ]:
# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best4 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best4.fit(train_x, train_y_delta)

xgb_train = xgb_best4.predict(train_x)
xgb_test = xgb_best4.predict(test_x)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn4.index, surface_tp_train_hybrid_stn4.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, xgb_train, label="train")

plt.plot(test_x.index, xgb_test, label="train")

In [ ]:
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

In [ ]:
plt.plot(surface_tp_train_hybrid_stn4.index, surface_tp_train_hybrid_stn4.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, original_train, label="train")

plt.plot(test_x.index, original_test, label="train")

### 지점5

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn5 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 5].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn5.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn5.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn5 = pd.get_dummies(data=surface_tp_train_hybrid_stn5, drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn5[f_col],
                                                    surface_tp_train_hybrid_stn5[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm5 = LinearRegression(fit_intercept=False)
lm5.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm5.predict(X)
trend_test = lm5.predict(X_)

plt.plot(surface_tp_train_hybrid_stn5.index, surface_tp_train_hybrid_stn5.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, trend_train, label="train")

plt.plot(test_x.index, trend_test, label="train")

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [2]:
# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best5 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best5.fit(train_x, train_y_delta)

xgb_train = xgb_best5.predict(train_x)
xgb_test = xgb_best5.predict(test_x)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn5.index, surface_tp_train_hybrid_stn5.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, xgb_train, label="train")

plt.plot(test_x.index, xgb_test, label="train")

In [ ]:
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

plt.plot(surface_tp_train_hybrid_stn5.index, surface_tp_train_hybrid_stn5.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, original_train, label="train")

plt.plot(test_x.index, original_test, label="train")

### 지점6

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn6 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 6].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn6.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn6.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn6 = pd.get_dummies(data=surface_tp_train_hybrid_stn6, drop_first=True)

from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn6[f_col],
                                                    surface_tp_train_hybrid_stn6[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm6 = LinearRegression(fit_intercept=False)
lm6.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm6.predict(X)
trend_test = lm6.predict(X_)

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [ ]:
# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

In [ ]:
def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best6 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best6.fit(train_x, train_y_delta)

xgb_train = xgb_best6.predict(train_x)
xgb_test = xgb_best6.predict(test_x)

In [ ]:
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

### 지점 7 

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn7 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 7].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn7.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn7.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn7 = pd.get_dummies(data=surface_tp_train_hybrid_stn7, drop_first=True)

from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn7[f_col],
                                                    surface_tp_train_hybrid_stn7[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm7 = LinearRegression(fit_intercept=False)
lm7.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm7.predict(X)
trend_test = lm7.predict(X_)

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

In [ ]:
def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best7 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best7.fit(train_x, train_y_delta)

xgb_train = xgb_best7.predict(train_x)
xgb_test = xgb_best7.predict(test_x)

plt.plot(surface_tp_train_hybrid_stn7.index, surface_tp_train_hybrid_stn7.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, xgb_train, label="train")

plt.plot(test_x.index, xgb_test, label="train")

original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

plt.plot(surface_tp_train_hybrid_stn7.index, surface_tp_train_hybrid_stn7.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, original_train, label="train")

plt.plot(test_x.index, original_test, label="train")

### 지점 8 

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn8 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 8].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn8.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn8.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn8 = pd.get_dummies(data=surface_tp_train_hybrid_stn8, drop_first=True)

from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn8[f_col],
                                                    surface_tp_train_hybrid_stn8[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm8 = LinearRegression(fit_intercept=False)
lm8.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm8.predict(X)
trend_test = lm8.predict(X_)

In [ ]:
plt.plot(surface_tp_train_hybrid_stn8.index, surface_tp_train_hybrid_stn8.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, trend_train, label="train")

plt.plot(test_x.index, trend_test, label="train")

In [ ]:
import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

In [ ]:
# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

In [ ]:
def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

In [ ]:
def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
xgb_best8 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best8.fit(train_x, train_y_delta)

xgb_train = xgb_best8.predict(train_x)
xgb_test = xgb_best8.predict(test_x)

In [ ]:
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn9 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 9].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn9.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn9.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn9 = pd.get_dummies(data=surface_tp_train_hybrid_stn9, drop_first=True)

from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn9[f_col],
                                                    surface_tp_train_hybrid_stn9[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm9 = LinearRegression(fit_intercept=False)
lm9.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm9.predict(X)
trend_test = lm9.predict(X_)

import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# xgb= xgboost.XGBRegressor()
# xgb.fit(train_x, train_y_delta)

# xgb_train = xgb.predict(train_x)
# xgb_test = xgb.predict(test_x)

# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

xgb_best9 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best9.fit(train_x, train_y_delta)

xgb_train = xgb_best9.predict(train_x)
xgb_test = xgb_best9.predict(test_x)

original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]

plt.plot(surface_tp_train_hybrid_stn9.index, surface_tp_train_hybrid_stn9.loc[:, 'ts'], label="원본")

plt.plot(train_x.index, original_train, label="train")

plt.plot(test_x.index, original_test, label="train")



### 지점 10 

In [ ]:
surface_tp_train_hybrid = surface_tp_train.copy()

surface_tp_train_hybrid_stn10 = surface_tp_train_hybrid[surface_tp_train_hybrid.stn == 10].reset_index(drop=True)

start_date = "1900-02-01 00:00:00"  # 시작 날짜
end_date = "1905-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_train_hybrid_stn10.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_train_hybrid_stn10.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_train_hybrid_stn10 = pd.get_dummies(data=surface_tp_train_hybrid_stn10, drop_first=True)

from sklearn.model_selection import train_test_split

target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

train_x, test_x, train_y, test_y = train_test_split(
                                                    surface_tp_train_hybrid_stn10[f_col],
                                                    surface_tp_train_hybrid_stn10[target],
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    )

from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression

dp = DeterministicProcess(
                          index = train_x.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()
lm10 = LinearRegression(fit_intercept=False)
lm10.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
# test_trend_squared = [i**2 for i in test_trend]
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm10.predict(X)
trend_test = lm10.predict(X_)

import xgboost

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# model
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# misc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.metrics import mean_absolute_error

def XGB_params_fold_start(XGB_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = xgboost.XGBRegressor(**XGB_params)

        model.fit(x_train, y_train,
                eval_set=[(x_val,y_val)],
                eval_metric='mae', verbose=False, early_stopping_rounds=100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)

        mae_list.append(result)

    return np.mean(mae_list)

def objectiveXGB(trial: Trial, X, y):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' : 0.01,
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method' : 'gpu_hist',
        'predictor' : 'gpu_predictor',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : 1127,
        'objective' : 'reg:absoluteerror'
    }

    score = XGB_params_fold_start(param, X, y)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

study.optimize(lambda trial : objectiveXGB(trial, train_x, train_y_delta), n_trials=30)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

xgb_best10 = xgboost.XGBRegressor(**study.best_trial.params, **{'tree_method' : 'gpu_hist', 'predictor' : 'gpu_predictor'})
xgb_best10.fit(train_x, train_y_delta)

xgb_train = xgb_best10.predict(train_x)
xgb_test = xgb_best10.predict(test_x)

original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), xgb_train.tolist())]

original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), xgb_test.tolist())]



## 앙상블

In [ ]:
surface_tp_test_hybrid = surface_tp_test.copy()

# 기상청 설명자료 속 선행지식 활용 (지점 abc)
surface_tp_test_hybrid_stn1 = surface_tp_test_hybrid[surface_tp_test_hybrid.stn == 'a'].reset_index(drop=True)
surface_tp_test_hybrid_stn2 = surface_tp_test_hybrid[surface_tp_test_hybrid.stn == 'b'].reset_index(drop=True)
surface_tp_test_hybrid_stn3 = surface_tp_test_hybrid[surface_tp_test_hybrid.stn == 'c'].reset_index(drop=True)

# 기상청 설명자료 속 선행지식 활용 (F~G년)
start_date = "1906-02-01 00:00:00"  # 시작 날짜
end_date = "1907-01-31 23:00:00"  # 종료 날짜
freq = "H"  # 월 단위로 설정

surface_tp_test_hybrid_stn1.index = pd.date_range(start=start_date, end=end_date, freq=freq)
surface_tp_test_hybrid_stn2.index = pd.date_range(start=start_date, end=end_date, freq=freq)
surface_tp_test_hybrid_stn3.index = pd.date_range(start=start_date, end=end_date, freq=freq)

surface_tp_test_hybrid_stn1.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)
surface_tp_test_hybrid_stn2.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)
surface_tp_test_hybrid_stn3.drop(['stn', 'mmddhh', 'year'], axis=1, inplace=True)

surface_tp_test_hybrid_stn1 = pd.get_dummies(data=surface_tp_test_hybrid_stn1, drop_first=True)
surface_tp_test_hybrid_stn2 = pd.get_dummies(data=surface_tp_test_hybrid_stn2, drop_first=True)
surface_tp_test_hybrid_stn3 = pd.get_dummies(data=surface_tp_test_hybrid_stn3, drop_first=True)

In [ ]:
dp = DeterministicProcess(
                          index = surface_tp_train_hybrid_stn1.index,
                          constant = True,
                          order = 1,
                          drop = True,
                          )

X = dp.in_sample()

In [ ]:
target = ['ts']
f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F',
       'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

lm1.fit(X, surface_tp_train_hybrid_stn1[target])
lm2.fit(X, surface_tp_train_hybrid_stn2[target])
lm3.fit(X, surface_tp_train_hybrid_stn3[target])
lm4.fit(X, surface_tp_train_hybrid_stn4[target])
lm5.fit(X, surface_tp_train_hybrid_stn5[target])
lm6.fit(X, surface_tp_train_hybrid_stn6[target])
lm7.fit(X, surface_tp_train_hybrid_stn7[target])
lm8.fit(X, surface_tp_train_hybrid_stn8[target])
lm9.fit(X, surface_tp_train_hybrid_stn9[target])
lm10.fit(X, surface_tp_train_hybrid_stn10[target])

In [ ]:
# 지점별 train data 추세 추출
trend_stn1 = lm1.predict(X)
trend_stn2 = lm2.predict(X)
trend_stn3 = lm3.predict(X)
trend_stn4 = lm4.predict(X)
trend_stn5 = lm5.predict(X)
trend_stn6 = lm6.predict(X)
trend_stn7 = lm7.predict(X)
trend_stn8 = lm8.predict(X)
trend_stn9 = lm9.predict(X)
trend_stn10 = lm10.predict(X)

In [ ]:
# 추세를 제거한 계절성만 남은 전체 train data로 학습
xgb_best1.fit(surface_tp_train_hybrid_stn1[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn1[target].ts.tolist(),  sum(trend_stn1.tolist(), []))])
xgb_best2.fit(surface_tp_train_hybrid_stn2[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn2[target].ts.tolist(),  sum(trend_stn2.tolist(), []))])
xgb_best3.fit(surface_tp_train_hybrid_stn3[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn3[target].ts.tolist(),  sum(trend_stn3.tolist(), []))])
xgb_best4.fit(surface_tp_train_hybrid_stn4[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn4[target].ts.tolist(),  sum(trend_stn4.tolist(), []))])
xgb_best5.fit(surface_tp_train_hybrid_stn5[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn5[target].ts.tolist(),  sum(trend_stn5.tolist(), []))])
xgb_best6.fit(surface_tp_train_hybrid_stn6[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn6[target].ts.tolist(),  sum(trend_stn6.tolist(), []))])
xgb_best7.fit(surface_tp_train_hybrid_stn7[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn7[target].ts.tolist(),  sum(trend_stn7.tolist(), []))])
xgb_best8.fit(surface_tp_train_hybrid_stn8[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn8[target].ts.tolist(),  sum(trend_stn8.tolist(), []))])
xgb_best9.fit(surface_tp_train_hybrid_stn9[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn9[target].ts.tolist(),  sum(trend_stn9.tolist(), []))])
xgb_best10.fit(surface_tp_train_hybrid_stn10[f_col], [i-j for i,j in zip(surface_tp_train_hybrid_stn10[target].ts.tolist(),  sum(trend_stn10.tolist(), []))])

In [ ]:
# 없는 열은 0으로 설정
surface_tp_test_hybrid_stn1['ww_X'] = 0
surface_tp_test_hybrid_stn2['ww_X'] = 0
surface_tp_test_hybrid_stn3['ww_X'] = 0

surface_tp_test_hybrid_stn3['ww_H'] = 0

In [ ]:
# 지점 a
# a_trend_lm1 = lm1.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm2 = lm2.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm3 = lm3.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm4 = lm4.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm5 = lm5.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm6 = lm6.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm7 = lm7.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm8 = lm8.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm9 = lm9.predict(surface_tp_test_hybrid_stn1[f_col])
# a_trend_lm10 = lm10.predict(surface_tp_test_hybrid_stn1[f_col])

a_seasonality_xgb_best1 = xgb_best1.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best2 = xgb_best2.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best3 = xgb_best3.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best4 = xgb_best4.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best5 = xgb_best5.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best6 = xgb_best6.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best7 = xgb_best7.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best8 = xgb_best8.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best9 = xgb_best9.predict(surface_tp_test_hybrid_stn1[f_col])
a_seasonality_xgb_best10 = xgb_best10.predict(surface_tp_test_hybrid_stn1[f_col])

# 지점 b
# b_trend_lm1 = lm1.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm2 = lm2.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm3 = lm3.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm4 = lm4.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm5 = lm5.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm6 = lm6.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm7 = lm7.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm8 = lm8.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm9 = lm9.predict(surface_tp_test_hybrid_stn2[f_col])
# b_trend_lm10 = lm10.predict(surface_tp_test_hybrid_stn2[f_col])

b_seasonality_xgb_best1 = xgb_best1.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best2 = xgb_best2.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best3 = xgb_best3.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best4 = xgb_best4.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best5 = xgb_best5.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best6 = xgb_best6.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best7 = xgb_best7.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best8 = xgb_best8.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best9 = xgb_best9.predict(surface_tp_test_hybrid_stn2[f_col])
b_seasonality_xgb_best10 = xgb_best10.predict(surface_tp_test_hybrid_stn2[f_col])

# 지점 c
# c_trend_lm1 = lm1.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm2 = lm2.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm3 = lm3.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm4 = lm4.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm5 = lm5.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm6 = lm6.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm7 = lm7.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm8 = lm8.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm9 = lm9.predict(surface_tp_test_hybrid_stn3[f_col])
# c_trend_lm10 = lm10.predict(surface_tp_test_hybrid_stn3[f_col])

c_seasonality_xgb_best1 = xgb_best1.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best2 = xgb_best2.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best3 = xgb_best3.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best4 = xgb_best4.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best5 = xgb_best5.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best6 = xgb_best6.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best7 = xgb_best7.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best8 = xgb_best8.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best9 = xgb_best9.predict(surface_tp_test_hybrid_stn3[f_col])
c_seasonality_xgb_best10 = xgb_best10.predict(surface_tp_test_hybrid_stn3[f_col])

In [ ]:
# 10개 지점 추세 평균값을 사용
avg_trend = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(sum(trend_stn1.tolist(), []), sum(trend_stn2.tolist(), []), sum(trend_stn3.tolist(), []),
                                    sum(trend_stn4.tolist(), []), sum(trend_stn5.tolist(), []), sum(trend_stn6.tolist(), []),
                                    sum(trend_stn7.tolist(), []), sum(trend_stn8.tolist(), []), sum(trend_stn9.tolist(), []),
                                    sum(trend_stn10.tolist(), []))]

# a_trend = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(sum(a_trend_lm1.tolist(), []), sum(a_trend_lm2.tolist(), []), sum(a_trend_lm3.tolist(), []),
#                                     sum(a_trend_lm4.tolist(), []), sum(a_trend_lm5.tolist(), []), sum(a_trend_lm6.tolist(), []),
#                                     sum(a_trend_lm7.tolist(), []), sum(a_trend_lm8.tolist(), []), sum(a_trend_lm9.tolist(), []),
#                                     sum(a_trend_lm10.tolist(), []))]

a_seasonality = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(a_seasonality_xgb_best1.tolist(),a_seasonality_xgb_best2.tolist(), a_seasonality_xgb_best3.tolist(),
                                                                         a_seasonality_xgb_best4.tolist(),a_seasonality_xgb_best5.tolist(),a_seasonality_xgb_best6.tolist(),
                                                                         a_seasonality_xgb_best7.tolist(),a_seasonality_xgb_best8.tolist(),a_seasonality_xgb_best9.tolist(),
                                                                         a_seasonality_xgb_best10.tolist())]

a_pred = [i+j for i,j in zip(avg_trend, a_seasonality)]

# b_trend = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(sum(b_trend_lm1.tolist(), []), sum(b_trend_lm2.tolist(), []), sum(b_trend_lm3.tolist(), []),
#                                     sum(b_trend_lm4.tolist(), []), sum(b_trend_lm5.tolist(), []), sum(b_trend_lm6.tolist(), []),
#                                     sum(b_trend_lm7.tolist(), []), sum(b_trend_lm8.tolist(), []), sum(b_trend_lm9.tolist(), []),
#                                     sum(b_trend_lm10.tolist(), []))]

b_seasonality = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(b_seasonality_xgb_best1.tolist(),b_seasonality_xgb_best2.tolist(), b_seasonality_xgb_best3.tolist(),
                                                                         b_seasonality_xgb_best4.tolist(),b_seasonality_xgb_best5.tolist(), b_seasonality_xgb_best6.tolist(),
                                                                         b_seasonality_xgb_best7.tolist(),b_seasonality_xgb_best8.tolist(), b_seasonality_xgb_best9.tolist(),
                                                                         b_seasonality_xgb_best10.tolist())]

b_pred = [i+j for i,j in zip(avg_trend, b_seasonality)]

# c_trend = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(sum(c_trend_lm1.tolist(), []), sum(c_trend_lm2.tolist(), []), sum(c_trend_lm3.tolist(), []),
#                                     sum(c_trend_lm4.tolist(), []), sum(c_trend_lm5.tolist(), []), sum(c_trend_lm6.tolist(), []),
#                                     sum(c_trend_lm7.tolist(), []), sum(c_trend_lm8.tolist(), []), sum(c_trend_lm9.tolist(), []),
#                                     sum(c_trend_lm10.tolist(), []))]

c_seasonality = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(c_seasonality_xgb_best1.tolist(),c_seasonality_xgb_best2.tolist(), c_seasonality_xgb_best3.tolist(),
                                                                         c_seasonality_xgb_best4.tolist(),c_seasonality_xgb_best5.tolist(), c_seasonality_xgb_best6.tolist(),
                                                                         c_seasonality_xgb_best7.tolist(),c_seasonality_xgb_best8.tolist(), c_seasonality_xgb_best9.tolist(),
                                                                         c_seasonality_xgb_best10.tolist())]

c_pred = [i+j for i,j in zip(avg_trend, c_seasonality)]

In [ ]:
surface_tp_test = pd.read_csv('/content/drive/MyDrive/기상청/데이터/지면온도/surface_tp_test.csv')

# 불필요 첫 2열 제거
surface_tp_test = surface_tp_test.iloc[:, 1:]

# 예측값 병합
a_pred.extend(b_pred)

a_pred.extend(c_pred)

# 겨울 모델링 

In [ ]:
# Library
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import missingno as msno

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder , RobustScaler
from sklearn.metrics import mean_absolute_error

## CV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn import metrics

## Model
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.linear_model import LinearRegression
import xgboost
import lightgbm as lgb

## Tunning
import time
import re
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_fin = pd.read_csv('/content/drive/MyDrive/기상청/코드/이지윤/데이터/보간_수정_train.csv',encoding = 'UTF8')
test_fin = pd.read_csv('/content/drive/MyDrive/기상청/코드/이지윤/데이터/보간_수정_test.csv',encoding = 'UTF8')

In [ ]:
## Train Data
# Label Encoding
encoder = LabelEncoder()
encoder.fit(train_fin['year'])
train_fin["year"] = encoder.transform(train_fin['year']) + 1

# month / day / hour
train_fin['mmddhh'] = train_fin['mmddhh'].astype('str')
train_fin['hour'] = train_fin['mmddhh'].str[-2:]
train_fin['day'] = train_fin['mmddhh'].str[-4:-2]
train_fin['month'] = train_fin['mmddhh'].str[:-4]

train_fin['datetime'] = '000' + train_fin['year'].astype(str) + '-' + train_fin['month'] + '-' + train_fin['day'] + ' ' + train_fin['hour'] + ':00:00'
train_fin.drop(['mmddhh','year'], axis = 1 , inplace = True)

train_fin['month'] = train_fin['month'].astype('int')

# Y 에 Na 존재하는거 제거
train_fin['month'] = train_fin['month'].astype(int)
train_fin.dropna(subset=['ts'], inplace = True)

# Date를 Axis로 지정
train_fin.drop(['hour','day','Unnamed: 0'], axis = 1 , inplace = True)
train_fin = train_fin.reset_index(drop = True).set_index('datetime')

In [ ]:
## Test Data
# Label Encoding
encoder = LabelEncoder()
encoder.fit(test_fin['year'])
test_fin["year"] = encoder.transform(test_fin['year']) + 1

encoder = LabelEncoder()
encoder.fit(test_fin['stn'])
test_fin['stn'] = encoder.transform(test_fin['stn']) + 1

# month / day / hour
test_fin['mmddhh'] = test_fin['mmddhh'].astype('str')
test_fin['hour'] = test_fin['mmddhh'].str[-2:]
test_fin['day'] = test_fin['mmddhh'].str[-4:-2]
test_fin['month'] = test_fin['mmddhh'].str[:-4]

test_fin['datetime'] = '000' + test_fin['year'].astype(str) + '-' + test_fin['month'] + '-' + test_fin['day'] + ' ' + test_fin['hour'] + ':00:00'

# Date를 Axis로 지정
test_fin.drop(['Unnamed: 0','mmddhh','year','hour','day','month'], axis = 1 , inplace = True)
test_fin = test_fin.reset_index(drop = True).set_index('datetime')

In [ ]:
train_hybrid = train_fin.copy()

In [ ]:
def LGBM_params_fold_start(LGBM_params, X, y):

    tscv = TimeSeriesSplit(n_splits=5)

    pred_list = []
    mae_list = []

    for train_index, val_index in tscv.split(X):

        x_train, x_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]

        model = lgb.LGBMRegressor(**LGBM_params)
        model.fit(x_train , y_train ,
                eval_set=[(x_val,y_val)],
                eval_metric = 'mae' , verbose = False , early_stopping_rounds = 100)

        pred = model.predict(x_val)
        result = mean_absolute_error(pred,y_val)
        mae_list.append(result)

    return np.mean(mae_list)

In [ ]:
def objectiveLGB(trial: Trial, X, y):
    param_lgb = {
    'learning_rate': trial.suggest_float('learning_rate', 0.15, 0.2),
    'n_estimators': trial.suggest_categorical('n_estimators', [700 , 800 , 900]),
    'max_depth': trial.suggest_categorical('max_depth', [14, 15, 16, 17, 18]),
    'min_child_weight': trial.suggest_categorical('min_child_weight', [1 , 2, 3, 4]),
    'reg_lambda': trial.suggest_float('l2_regularization', 0, 1),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
    'num_leaves': trial.suggest_int('num_leaves', 20, 30,),
    'min_split_gain': trial.suggest_float('min_split_gain', 0, 1),
    'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
    'random_state': 112,
    'objective': 'huber',
    'metric': 'mae'
  }
    score = LGBM_params_fold_start(param_lgb, X, y)

    return score

In [ ]:
## STN 1
# Data
train_hybrid_stn1 = train_hybrid[train_hybrid.stn == 1].reset_index(drop = True)
train_hybrid_stn1 = pd.get_dummies(data = train_hybrid_stn1, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn1[f_col] , train_hybrid_stn1[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm1 = LinearRegression(fit_intercept=False)
lm1.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train = lm1.predict(X)
trend_test = lm1.predict(X_)

train_y_delta = train_y - trend_train
test_y_delta = test_y - trend_test

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42))

timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials= 1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param1 = study.best_trial.params
lgb_best1 = lgb.LGBMRegressor(**lgb_param1)
lgb_best1.fit(train_x, train_y_delta)

lgb_train = lgb_best1.predict(train_x)
lgb_test = lgb_best1.predict(test_x)

# Result
original_train = [i+j for i,j in zip(sum(trend_train.tolist(), []), lgb_train.tolist())]
original_test = [i+j for i,j in zip(sum(trend_test.tolist(), []), lgb_test.tolist())]

mean_absolute_error(original_test,test_y)

In [ ]:
## STN 2
# Data
train_hybrid_stn2 = train_hybrid[train_hybrid.stn == 2].reset_index(drop = True)
train_hybrid_stn2 = pd.get_dummies(data = train_hybrid_stn2, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn2[f_col] , train_hybrid_stn2[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm2 = LinearRegression(fit_intercept=False)
lm2.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train2 = lm2.predict(X)
trend_test2 = lm2.predict(X_)

train_y_delta = train_y - trend_train2
test_y_delta = test_y - trend_test2

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param2 = study.best_trial.params
lgb_best2 = lgb.LGBMRegressor(**lgb_param2)
lgb_best2.fit(train_x, train_y_delta)

lgb_train2 = lgb_best2.predict(train_x)
lgb_test2 = lgb_best2.predict(test_x)

# Result
original_train2 = [i+j for i,j in zip(sum(trend_train2.tolist(), []), lgb_train2.tolist())]
original_test2 = [i+j for i,j in zip(sum(trend_test2.tolist(), []), lgb_test2.tolist())]

mean_absolute_error(original_test2 , test_y)

In [ ]:
## STN 3
# Data
train_hybrid_stn3 = train_hybrid[train_hybrid.stn == 3].reset_index(drop = True)
train_hybrid_stn3 = pd.get_dummies(data = train_hybrid_stn3, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn3[f_col] , train_hybrid_stn3[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm3 = LinearRegression(fit_intercept=False)
lm3.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train3 = lm3.predict(X)
trend_test3 = lm3.predict(X_)

train_y_delta = train_y - trend_train3
test_y_delta = test_y - trend_test3

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param3 = study.best_trial.params
lgb_best3 = lgb.LGBMRegressor(**lgb_param3)
lgb_best3.fit(train_x, train_y_delta)

lgb_train3 = lgb_best3.predict(train_x)
lgb_test3 = lgb_best3.predict(test_x)

# Result
original_train3 = [i+j for i,j in zip(sum(trend_train3.tolist(), []), lgb_train3.tolist())]
original_test3 = [i+j for i,j in zip(sum(trend_test3.tolist(), []), lgb_test3.tolist())]

mean_absolute_error(original_test3 , test_y)

In [ ]:
## STN 4
# Data
train_hybrid_stn4 = train_hybrid[train_hybrid.stn == 4].reset_index(drop = True)
train_hybrid_stn4 = pd.get_dummies(data = train_hybrid_stn4, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn4[f_col] , train_hybrid_stn4[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm4 = LinearRegression(fit_intercept=False)
lm4.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train4 = lm4.predict(X)
trend_test4 = lm4.predict(X_)

train_y_delta = train_y - trend_train4
test_y_delta = test_y - trend_test4

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param4 = study.best_trial.params
lgb_best4 = lgb.LGBMRegressor(**lgb_param4)
lgb_best4.fit(train_x, train_y_delta)

lgb_train4 = lgb_best4.predict(train_x)
lgb_test4 = lgb_best4.predict(test_x)

# Result
original_train4 = [i+j for i,j in zip(sum(trend_train4.tolist(), []), lgb_train4.tolist())]
original_test4 = [i+j for i,j in zip(sum(trend_test4.tolist(), []), lgb_test4.tolist())]

mean_absolute_error(original_test4 , test_y)

In [ ]:
## STN 5
# Data
train_hybrid_stn5 = train_hybrid[train_hybrid.stn == 5].reset_index(drop = True)
train_hybrid_stn5 = pd.get_dummies(data = train_hybrid_stn5, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn5[f_col] , train_hybrid_stn5[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm5 = LinearRegression(fit_intercept=False)
lm5.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train5 = lm5.predict(X)
trend_test5 = lm5.predict(X_)

train_y_delta = train_y - trend_train5
test_y_delta = test_y - trend_test5

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param5 = study.best_trial.params
lgb_best5 = lgb.LGBMRegressor(**lgb_param5)
lgb_best5.fit(train_x, train_y_delta)

lgb_train5 = lgb_best5.predict(train_x)
lgb_test5 = lgb_best5.predict(test_x)

# Result
original_train5 = [i+j for i,j in zip(sum(trend_train5.tolist(), []), lgb_train5.tolist())]
original_test5 = [i+j for i,j in zip(sum(trend_test5.tolist(), []), lgb_test5.tolist())]

mean_absolute_error(original_test5 , test_y)

In [ ]:
## STN 6
# Data
train_hybrid_stn6 = train_hybrid[train_hybrid.stn == 6].reset_index(drop = True)
train_hybrid_stn6 = pd.get_dummies(data = train_hybrid_stn6, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn6[f_col] , train_hybrid_stn6[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm6 = LinearRegression(fit_intercept=False)
lm6.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train6 = lm6.predict(X)
trend_test6 = lm6.predict(X_)

train_y_delta = train_y - trend_train6
test_y_delta = test_y - trend_test6

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param6 = study.best_trial.params
lgb_best6 = lgb.LGBMRegressor(**lgb_param6)
lgb_best6.fit(train_x, train_y_delta)

lgb_train6 = lgb_best6.predict(train_x)
lgb_test6 = lgb_best6.predict(test_x)

# Result
original_train6 = [i+j for i,j in zip(sum(trend_train6.tolist(), []), lgb_train6.tolist())]
original_test6 = [i+j for i,j in zip(sum(trend_test6.tolist(), []), lgb_test6.tolist())]

mean_absolute_error(original_test6 , test_y)

In [ ]:
## STN 7
# Data
train_hybrid_stn7 = train_hybrid[train_hybrid.stn == 7].reset_index(drop = True)
train_hybrid_stn7 = pd.get_dummies(data = train_hybrid_stn7, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn7[f_col] , train_hybrid_stn7[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm7 = LinearRegression(fit_intercept=False)
lm7.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train7 = lm7.predict(X)
trend_test7 = lm7.predict(X_)

train_y_delta = train_y - trend_train7
test_y_delta = test_y - trend_test7

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param7 = study.best_trial.params
lgb_best7 = lgb.LGBMRegressor(**lgb_param7)
lgb_best7.fit(train_x, train_y_delta)

lgb_train7 = lgb_best7.predict(train_x)
lgb_test7 = lgb_best7.predict(test_x)

# Result
original_train7 = [i+j for i,j in zip(sum(trend_train7.tolist(), []), lgb_train7.tolist())]
original_test7 = [i+j for i,j in zip(sum(trend_test7.tolist(), []), lgb_test7.tolist())]

mean_absolute_error(original_test7 , test_y)

In [ ]:
## STN 8
# Data
train_hybrid_stn8 = train_hybrid[train_hybrid.stn == 8].reset_index(drop = True)
train_hybrid_stn8 = pd.get_dummies(data = train_hybrid_stn8, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn8[f_col] , train_hybrid_stn8[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm8 = LinearRegression(fit_intercept=False)
lm8.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train8 = lm8.predict(X)
trend_test8 = lm8.predict(X_)

train_y_delta = train_y - trend_train8
test_y_delta = test_y - trend_test8

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param8 = study.best_trial.params
lgb_best8 = lgb.LGBMRegressor(**lgb_param8)
lgb_best8.fit(train_x, train_y_delta)

lgb_train8 = lgb_best8.predict(train_x)
lgb_test8 = lgb_best8.predict(test_x)

# Result
original_train8 = [i+j for i,j in zip(sum(trend_train8.tolist(), []), lgb_train8.tolist())]
original_test8 = [i+j for i,j in zip(sum(trend_test8.tolist(), []), lgb_test8.tolist())]

mean_absolute_error(original_test8 , test_y)

In [ ]:
## STN 9
# Data
train_hybrid_stn9 = train_hybrid[train_hybrid.stn == 9].reset_index(drop = True)
train_hybrid_stn9 = pd.get_dummies(data = train_hybrid_stn9, drop_first = True)

# train-test split
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn9[f_col] , train_hybrid_stn9[target] , test_size = 0.2 , shuffle = False)

# Time series
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm9 = LinearRegression(fit_intercept=False)
lm9.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max() + test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train9 = lm9.predict(X)
trend_test9 = lm9.predict(X_)

train_y_delta = train_y - trend_train9
test_y_delta = test_y - trend_test9

# Optuna tunning for tree model (LGBM)
study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param9 = study.best_trial.params
lgb_best9 = lgb.LGBMRegressor(**lgb_param9)
lgb_best9.fit(train_x, train_y_delta)

lgb_train9 = lgb_best9.predict(train_x)
lgb_test9 = lgb_best9.predict(test_x)

# Result
original_train9 = [i+j for i,j in zip(sum(trend_train9.tolist(), []), lgb_train9.tolist())]
original_test9 = [i+j for i,j in zip(sum(trend_test9.tolist(), []), lgb_test9.tolist())]

mean_absolute_error(original_test9 , test_y)

In [ ]:
## STN 10
train_hybrid_stn10 = train_hybrid[train_hybrid.stn == 10].reset_index(drop = True)
train_hybrid_stn10 = pd.get_dummies(data = train_hybrid_stn10, drop_first = True)

target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn10[f_col] , train_hybrid_stn10[target] , test_size = 0.2 , shuffle = False)

dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)

X = dp.in_sample()
lm10 = LinearRegression(fit_intercept=False)
lm10.fit(X, train_y)

test_const = [1 for i in range(test_x.shape[0])]
test_trend = np.arange(X['trend'].max(), X['trend'].max()+test_x.shape[0])
X_ = pd.DataFrame([test_const, test_trend]).T
X_.columns = ['const', 'trend']
X_.index = test_x.index

trend_train10 = lm10.predict(X)
trend_test10 = lm10.predict(X_)

train_y_delta = train_y - trend_train10
test_y_delta = test_y - trend_test10

study = optuna.create_study(direction='minimize',sampler = TPESampler(seed=42))
timeout = 1800 ; start_time = time.time()

while (time.time() - start_time) < timeout:
    study.optimize(lambda trial : objectiveLGB(trial, train_x, train_y_delta), n_trials=1)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

    if (time.time() - start_time) >= timeout:
        break

lgb_param10 = study.best_trial.params
lgb_best10 = lgb.LGBMRegressor(**lgb_param10)
lgb_best10.fit(train_x, train_y_delta)

lgb_train10 = lgb_best10.predict(train_x)
lgb_test10 = lgb_best10.predict(test_x)

original_train10 = [i+j for i,j in zip(sum(trend_train.tolist(), []), lgb_train10.tolist())]
original_test10 = [i+j for i,j in zip(sum(trend_test.tolist(), []), lgb_test10.tolist())]

mean_absolute_error(original_test10 , test_y)

In [ ]:
surface_tp_test_hybrid = test_fin.copy()

In [ ]:
# 기상청 설명자료 속 선행지식 활용 (지점 abc)
test_hybrid_stn1 = surface_tp_test_hybrid[surface_tp_test_hybrid.stn == 1].reset_index(drop=True)
test_hybrid_stn2 = surface_tp_test_hybrid[surface_tp_test_hybrid.stn == 2].reset_index(drop=True)
test_hybrid_stn3 = surface_tp_test_hybrid[surface_tp_test_hybrid.stn == 3].reset_index(drop=True)

test_hybrid_stn1 = pd.get_dummies(data=test_hybrid_stn1, drop_first=True)
test_hybrid_stn2 = pd.get_dummies(data=test_hybrid_stn2, drop_first=True)
test_hybrid_stn3 = pd.get_dummies(data=test_hybrid_stn3, drop_first=True)

In [ ]:
# dummy encoding을 했을 때 train에는 존재하지만, test에는 없는 열은 0으로 설정
test_hybrid_stn1['ww_X'] = 0
test_hybrid_stn2['ww_X'] = 0
test_hybrid_stn3['ww_X'] = 0
test_hybrid_stn3['ww_H'] = 0

In [ ]:
### Time Series
target = ['ts'] ; f_col = ['ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'sn', 'ww_F', 'ww_G', 'ww_H', 'ww_R', 'ww_S', 'ww_X']

## STN 1
train_hybrid_stn1 = train_hybrid[train_hybrid.stn == 1].reset_index(drop = True)
train_hybrid_stn1 = pd.get_dummies(data = train_hybrid_stn1, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn1[f_col] , train_hybrid_stn1[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X1 = dp.in_sample()
lm1 = LinearRegression(fit_intercept=False)

## STN 2
train_hybrid_stn2 = train_hybrid[train_hybrid.stn == 2].reset_index(drop = True)
train_hybrid_stn2 = pd.get_dummies(data = train_hybrid_stn2, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn2[f_col] , train_hybrid_stn2[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X2 = dp.in_sample()
lm2 = LinearRegression(fit_intercept=False)

## STN 3
train_hybrid_stn3 = train_hybrid[train_hybrid.stn == 3].reset_index(drop = True)
train_hybrid_stn3 = pd.get_dummies(data = train_hybrid_stn3, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn3[f_col] , train_hybrid_stn3[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X3 = dp.in_sample()
lm3 = LinearRegression(fit_intercept=False)

## STN 4
train_hybrid_stn4 = train_hybrid[train_hybrid.stn == 4].reset_index(drop = True)
train_hybrid_stn4 = pd.get_dummies(data = train_hybrid_stn4, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn4[f_col] , train_hybrid_stn4[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X4 = dp.in_sample()
lm4 = LinearRegression(fit_intercept=False)

## STN 5
train_hybrid_stn5 = train_hybrid[train_hybrid.stn == 5].reset_index(drop = True)
train_hybrid_stn5 = pd.get_dummies(data = train_hybrid_stn2, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn5[f_col] , train_hybrid_stn5[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X5 = dp.in_sample()
lm5 = LinearRegression(fit_intercept=False)

## STN 6
train_hybrid_stn6 = train_hybrid[train_hybrid.stn == 6].reset_index(drop = True)
train_hybrid_stn6 = pd.get_dummies(data = train_hybrid_stn6, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn6[f_col] , train_hybrid_stn6[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X6 = dp.in_sample()
lm6 = LinearRegression(fit_intercept=False)

## STN 7
train_hybrid_stn7 = train_hybrid[train_hybrid.stn == 7].reset_index(drop = True)
train_hybrid_stn7 = pd.get_dummies(data = train_hybrid_stn7, drop_first = True)
train_hybrid_stn7['ww_X'] = 0
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn7[f_col] , train_hybrid_stn7[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X7 = dp.in_sample()
lm7 = LinearRegression(fit_intercept=False)

## STN 8
train_hybrid_stn8 = train_hybrid[train_hybrid.stn == 8].reset_index(drop = True)
train_hybrid_stn8 = pd.get_dummies(data = train_hybrid_stn8, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn8[f_col] , train_hybrid_stn8[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X8 = dp.in_sample()
lm8 = LinearRegression(fit_intercept=False)

## STN 9
train_hybrid_stn9 = train_hybrid[train_hybrid.stn == 9].reset_index(drop = True)
train_hybrid_stn9 = pd.get_dummies(data = train_hybrid_stn9, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn9[f_col] , train_hybrid_stn9[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X9 = dp.in_sample()
lm9 = LinearRegression(fit_intercept=False)

## STN 10
train_hybrid_stn10 = train_hybrid[train_hybrid.stn == 10].reset_index(drop = True)
train_hybrid_stn10 = pd.get_dummies(data = train_hybrid_stn2, drop_first = True)
train_x, test_x, train_y, test_y = train_test_split(train_hybrid_stn10[f_col] , train_hybrid_stn10[target] , test_size = 0.2 , shuffle = False)
dp = DeterministicProcess(index = train_x.index , constant = True , order = 1 , drop = True)
X10 = dp.in_sample()
lm10 = LinearRegression(fit_intercept=False)

In [ ]:
# dp
dp1 = DeterministicProcess(index = train_hybrid_stn1.index, constant = True, order = 1,drop = True)
dp2 = DeterministicProcess(index = train_hybrid_stn2.index, constant = True, order = 1,drop = True)
dp3 = DeterministicProcess(index = train_hybrid_stn3.index, constant = True, order = 1,drop = True)
dp4 = DeterministicProcess(index = train_hybrid_stn4.index, constant = True, order = 1,drop = True)
dp5 = DeterministicProcess(index = train_hybrid_stn5.index, constant = True, order = 1,drop = True)
dp6 = DeterministicProcess(index = train_hybrid_stn6.index, constant = True, order = 1,drop = True)
dp7 = DeterministicProcess(index = train_hybrid_stn7.index, constant = True, order = 1,drop = True)
dp8 = DeterministicProcess(index = train_hybrid_stn8.index, constant = True, order = 1,drop = True)
dp9 = DeterministicProcess(index = train_hybrid_stn9.index, constant = True, order = 1,drop = True)
dp10 = DeterministicProcess(index = train_hybrid_stn10.index, constant = True, order = 1,drop = True)

# X
X1 = dp1.in_sample() ; X2 = dp2.in_sample()
X3 = dp3.in_sample() ; X4 = dp4.in_sample()
X5 = dp5.in_sample() ; X6 = dp6.in_sample()
X7 = dp7.in_sample() ; X8 = dp8.in_sample()
X9 = dp9.in_sample() ; X10 = dp10.in_sample()

In [ ]:
lm1.fit(X1, train_hybrid_stn1[target])
lm2.fit(X2, train_hybrid_stn2[target])
lm3.fit(X3, train_hybrid_stn3[target])
lm4.fit(X4, train_hybrid_stn4[target])
lm5.fit(X5, train_hybrid_stn5[target])
lm6.fit(X6, train_hybrid_stn6[target])
lm7.fit(X7, train_hybrid_stn7[target])
lm8.fit(X8, train_hybrid_stn8[target])
lm9.fit(X9, train_hybrid_stn9[target])
lm10.fit(X10, train_hybrid_stn10[target])

In [ ]:
# 지점별 train data 추세 추출
trend_stn1 = lm1.predict(X1)
trend_stn2 = lm2.predict(X2)
trend_stn3 = lm3.predict(X3)
trend_stn4 = lm4.predict(X4)
trend_stn5 = lm5.predict(X5)
trend_stn6 = lm6.predict(X6)
trend_stn7 = lm7.predict(X7)
trend_stn8 = lm8.predict(X8)
trend_stn9 = lm9.predict(X9)
trend_stn10 = lm10.predict(X10)

In [ ]:
# Best Parameter
lgb_param1 = {'learning_rate': 0.12829594410470951, 'n_estimators': 500, 'max_depth': 16, 'min_child_weight': 2, 'l2_regularization': 0.5336210596747876}
lgb_param2 = {'learning_rate': 0.16422823124652633, 'n_estimators': 800, 'max_depth': 16, 'min_child_weight': 2, 'l2_regularization': 0.48628837398667374,
              'colsample_bytree': 0.5877431614700145, 'num_leaves': 20, 'min_split_gain': 0.6322304426546818, 'bagging_freq': 10}
lgb_param3 = {'learning_rate': 0.18889534485982132, 'n_estimators': 500, 'max_depth': 17, 'min_child_weight': 4, 'l2_regularization': 0.32482046710162793}
lgb_param4 = {'learning_rate': 0.13046182446397234, 'n_estimators': 400, 'max_depth': 16, 'min_child_weight': 4, 'l2_regularization': 0.8668413429347246}
lgb_param5  = {'learning_rate': 0.1444166093465713,'n_estimators': 500,'max_depth': 17,'min_child_weight': 4, 'l2_regularization': 0.14729681542445355}
lgb_param6  = {'learning_rate': 0.16680093590253176 ,'n_estimators': 500, 'max_depth': 16, 'min_child_weight': 3,'l2_regularization': 0.5478551432621794}
lgb_param7  = {'learning_rate': 0.11803343495390418, 'n_estimators': 500, 'max_depth': 15, 'min_child_weight': 4, 'l2_regularization': 0.19592898855184526}
lgb_param8  ={'learning_rate': 0.12829594410470951, 'n_estimators': 500, 'max_depth': 16, 'min_child_weight': 2, 'l2_regularization': 0.5336210596747876}
lgb_param9  = {'learning_rate': 0.12829594410470951, 'n_estimators': 500, 'max_depth': 16, 'min_child_weight': 2, 'l2_regularization': 0.5336210596747876}
lgb_param10 = {'learning_rate': 0.11803433443831565,'n_estimators': 500,'max_depth': 17,'min_child_weight': 4, 'l2_regularization': 0.7771696259733507}

# Fit the model using best Parameter
lgb_best1 = lgb.LGBMRegressor(**lgb_param1)
lgb_best2 = lgb.LGBMRegressor(**lgb_param2)
lgb_best3 = lgb.LGBMRegressor(**lgb_param3)
lgb_best4 = lgb.LGBMRegressor(**lgb_param4)
lgb_best5 = lgb.LGBMRegressor(**lgb_param5)
lgb_best6 = lgb.LGBMRegressor(**lgb_param6)
lgb_best7 = lgb.LGBMRegressor(**lgb_param7)
lgb_best8 = lgb.LGBMRegressor(**lgb_param8)
lgb_best9 = lgb.LGBMRegressor(**lgb_param9)
lgb_best10 = lgb.LGBMRegressor(**lgb_param10)

In [ ]:
lgb_best1.fit(train_hybrid_stn1[f_col], [i-j for i,j in zip(train_hybrid_stn1[target].ts.tolist(),  sum(trend_stn1.tolist(), []))])
lgb_best2.fit(train_hybrid_stn2[f_col], [i-j for i,j in zip(train_hybrid_stn2[target].ts.tolist(),  sum(trend_stn2.tolist(), []))])
lgb_best3.fit(train_hybrid_stn3[f_col], [i-j for i,j in zip(train_hybrid_stn3[target].ts.tolist(),  sum(trend_stn3.tolist(), []))])
lgb_best4.fit(train_hybrid_stn4[f_col], [i-j for i,j in zip(train_hybrid_stn4[target].ts.tolist(),  sum(trend_stn4.tolist(), []))])
lgb_best5.fit(train_hybrid_stn5[f_col], [i-j for i,j in zip(train_hybrid_stn5[target].ts.tolist(),  sum(trend_stn5.tolist(), []))])
lgb_best6.fit(train_hybrid_stn6[f_col], [i-j for i,j in zip(train_hybrid_stn6[target].ts.tolist(),  sum(trend_stn6.tolist(), []))])
lgb_best7.fit(train_hybrid_stn7[f_col], [i-j for i,j in zip(train_hybrid_stn7[target].ts.tolist(),  sum(trend_stn7.tolist(), []))])
lgb_best8.fit(train_hybrid_stn8[f_col], [i-j for i,j in zip(train_hybrid_stn8[target].ts.tolist(),  sum(trend_stn8.tolist(), []))])
lgb_best9.fit(train_hybrid_stn9[f_col], [i-j for i,j in zip(train_hybrid_stn9[target].ts.tolist(),  sum(trend_stn9.tolist(), []))])
lgb_best10.fit(train_hybrid_stn10[f_col], [i-j for i,j in zip(train_hybrid_stn10[target].ts.tolist(),  sum(trend_stn10.tolist(), []))])

In [ ]:
# a년도의 지면온도를 stn 별로 예측
a_seasonality_lgb_best1 = lgb_best1.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best2 = lgb_best2.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best3 = lgb_best3.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best4 = lgb_best4.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best5 = lgb_best5.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best6 = lgb_best6.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best7 = lgb_best7.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best8 = lgb_best8.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best9 = lgb_best9.predict(test_hybrid_stn1[f_col])
a_seasonality_lgb_best10 = lgb_best10.predict(test_hybrid_stn1[f_col])

In [ ]:
# b년도의 지면온도를 stn 별로 예측
b_seasonality_lgb_best1 = lgb_best1.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best2 = lgb_best2.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best3 = lgb_best3.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best4 = lgb_best4.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best5 = lgb_best5.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best6 = lgb_best6.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best7 = lgb_best7.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best8 = lgb_best8.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best9 = lgb_best9.predict(test_hybrid_stn2[f_col])
b_seasonality_lgb_best10 = lgb_best10.predict(test_hybrid_stn2[f_col])

In [ ]:
# c년도의 지면온도를 stn 별로 예측
c_seasonality_lgb_best1 = lgb_best1.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best2 = lgb_best2.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best3 = lgb_best3.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best4 = lgb_best4.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best5 = lgb_best5.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best6 = lgb_best6.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best7 = lgb_best7.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best8 = lgb_best8.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best9 = lgb_best9.predict(test_hybrid_stn3[f_col])
c_seasonality_lgb_best10 = lgb_best10.predict(test_hybrid_stn3[f_col])

In [ ]:
# 10개 지점 추세 평균값을 사용
avg_trend = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(sum(trend_stn1.tolist(), []), sum(trend_stn2.tolist(), []), sum(trend_stn3.tolist(), []),
                                    sum(trend_stn4.tolist(), []), sum(trend_stn5.tolist(), []), sum(trend_stn6.tolist(), []),
                                    sum(trend_stn7.tolist(), []), sum(trend_stn8.tolist(), []), sum(trend_stn9.tolist(), []),
                                    sum(trend_stn10.tolist(), []))]

In [ ]:
a_seasonality = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(a_seasonality_lgb_best1.tolist(),a_seasonality_lgb_best2.tolist(), a_seasonality_lgb_best3.tolist(),
                                                                         a_seasonality_lgb_best4.tolist(),a_seasonality_lgb_best5.tolist(),a_seasonality_lgb_best6.tolist(),
                                                                         a_seasonality_lgb_best7.tolist(),a_seasonality_lgb_best8.tolist(),a_seasonality_lgb_best9.tolist(),
                                                                         a_seasonality_lgb_best10.tolist())]

a_pred = [i+j for i,j in zip(avg_trend, a_seasonality)]

In [ ]:
b_seasonality = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(b_seasonality_lgb_best1.tolist(),b_seasonality_lgb_best2.tolist(), b_seasonality_lgb_best3.tolist(),
                                                                         b_seasonality_lgb_best4.tolist(),b_seasonality_lgb_best5.tolist(), b_seasonality_lgb_best6.tolist(),
                                                                         b_seasonality_lgb_best7.tolist(),b_seasonality_lgb_best8.tolist(), b_seasonality_lgb_best9.tolist(),
                                                                         b_seasonality_lgb_best10.tolist())]

b_pred = [i+j for i,j in zip(avg_trend, b_seasonality)]

In [ ]:
c_seasonality = [(a+b+c+d+e+f+g+h+i+j)/10 for a,b,c,d,e,f,g,h,i,j in zip(c_seasonality_lgb_best1.tolist(),c_seasonality_lgb_best2.tolist(), c_seasonality_lgb_best3.tolist(),
                                                                         c_seasonality_lgb_best4.tolist(),c_seasonality_lgb_best5.tolist(), c_seasonality_lgb_best6.tolist(),
                                                                         c_seasonality_lgb_best7.tolist(),c_seasonality_lgb_best8.tolist(), c_seasonality_lgb_best9.tolist(),
                                                                         c_seasonality_lgb_best10.tolist())]

c_pred = [i+j for i,j in zip(avg_trend, c_seasonality)]

In [ ]:
# 예측한 ts 저장
a_pred.extend(b_pred)
a_pred.extend(c_pred)
test_fin['ts'] = a_pred

In [ ]:
# datetime에서 month 분리하기
test_fin.reset_index(inplace = True)
test_fin['month'] = test_fin['datetime'].str.split("-").str.get(1)
test_fin['month'] = test_fin['month'].astype(int)
test_fin.drop(['datetime'],axis = 1 , inplace = True)

In [ ]:
# 월 별로 나누기
winter = [11 , 12, 1] ; spring = [2, 3, 4] ; summer = [5, 6, 7]  ; autumn = [8 , 9, 10]

# Test
winter_test = test_fin[test_fin['month'].isin(winter)]
spring_test = test_fin[test_fin['month'].isin(spring)]
summer_test = test_fin[test_fin['month'].isin(summer)]
autumn_test = test_fin[test_fin['month'].isin(autumn)]